In [1]:
using DataFrames
using Chemfiles
using MIToS.PDB
using Gadfly
using Cairo
using Compose
set_default_plot_size(40cm, 14cm)

home = "/home/german/labo/18/egfr/cavidad"
home_modos = "/home/german/labo/18/egfr/modos"
home_rtdos = "/home/german/labo/18/egfr/rtdos_log"
aa = 277

INFO: Recompiling stale cache file /home/german/.julia/lib/v0.6/Requests.ji for module Requests.
INFO: Recompiling stale cache file /home/german/.julia/lib/v0.6/MIToS.ji for module MIToS.
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] @vectorize_1arg(::ANY, ::ANY) at ./deprecated.jl:322
 [3] include_from_node1(::String) at ./loading.jl:569
 [4] include(::String) at ./sysimg.jl:14
 [5] include_from_node1(::String) at ./loading.jl:569
 [6] include(::String) at ./sysimg.jl:14
 [7] anonymous at ./<missing>:2
 [8] eval(::Module, ::Any) at ./boot.jl:235
 [9] process_options(::Base.JLOptions) at ./client.jl:286
 [10] _start() at ./client.jl:371
while loading /home/german/.julia/v0.6/ROCAnalysis/src/modules.jl, in expression starting on line 14
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] @vectorize_1arg(::ANY, ::ANY) at ./deprecated.jl:322
 [3] include_from_node1(::String) at ./loading.jl:569
 [4] include(::String) at ./sysimg.jl:14
 [5] inc

277

In [2]:
function tognm(vtor_anm)
    vtor_gnm = Array{Float64}(convert(Int64, length(vtor_anm)/3));
    vtor_anm =  vtor_anm.^2
    for i=1:convert(Int64, length(vtor_anm)/3)
        vtor_gnm[i] = sqrt(vtor_anm[i*3-2] + vtor_anm[i*3-1] + vtor_anm[i*3])
    end
    return vtor_gnm
end

tognm (generic function with 1 method)

In [3]:
function WeightedHist(in_vec, in_bins, in_weight, density = false)
    # Safety check    
    if length(in_vec) != length(in_weight)
        println("Each element of the input vector needs one weight")
        return
    end
    
    # Prepare variables
    out_counts = Array{Float64}(length(in_bins)-1)
    
    # Get weighted histogram
    for i=1:length(in_bins)-1
        if i == 1
            # Include those that fall before the beggining of the bins
            temp_bool = (in_vec .>= in_bins[i]) .& (in_vec .< in_bins[i+1]) .| (in_vec .<= in_bins[i])
            out_counts[i] = sum(in_weight[temp_bool])
        elseif i == length(in_bins)-1
            # Include those that fall after the end of the bins
            temp_bool = (in_vec .>= in_bins[i]) .& (in_vec .< in_bins[i+1]) .| (in_vec .>= in_bins[end])
            out_counts[i] = sum(in_weight[temp_bool])
        else
            temp_bool = (in_vec .>= in_bins[i]) .& (in_vec .< in_bins[i+1])
            out_counts[i] = sum(in_weight[temp_bool])
        end
    end
    
    # Get bins middle points
    out_middle = (in_bins[1:end-1] + in_bins[2:end]) / 2
    
    # Turn counts into density
    if (density == true)
        out_counts = out_counts ./ sum(out_counts) 
    end
    return out_counts, out_middle
end

WeightedHist (generic function with 2 methods)

In [4]:
function displaceAA(mod_pdb, in_vector, multiplier)
    # Preparo variables
    pdb = copy(mod_pdb)
    struct_xyz = coordinatesmatrix(pdb)
    new_struct_xyz = copy(struct_xyz)
        aa = length(pdb)
        # Determino el nro de atomos de c/ aminoácido
        natom = Array{Int64}(aa)
        [ natom[i] = length(pdb[i]) for i = 1:aa ]
        # Adapto el vector p/ darle la misma forma q la matriz de coordenadas
        vector = Array{Float64}
        const tmp_size = size(in_vector)
        const natoms = sum(natom)

        if tmp_size == (aa*3, )
                vector = transpose(reshape(in_vector, 3, aa))
        elseif tmp_size == (aa, 3)
                vector = in_vector
        else
                error("Input vector with wrong dimensions: ", tmp_size, "  ", (aa*3, 1))
        end
        sum_mat = Array{Float64}(sum(natom),3)
        cursor = 0
        for i = 1:aa
                rango = Array{Int64}(natom[i])
        if i == 1
                        sum_mat[1:natom[i], :] = repmat(transpose(vector[i, :]),
                                natom[i], 1)
                        cursor = natom[i]
                        continue
                end
                rango = collect(cursor+1:cursor + natom[i])
                sum_mat[rango, :] = repmat(transpose(vector[i, :]), natom[i], 1)
                cursor += natom[i]
        end

   # Listo, ahora puedo mover el pdb
   new_struct_xyz  = struct_xyz + sum_mat .* multiplier
   pdb = change_coordinates(pdb, new_struct_xyz);
   return pdb
end

displaceAA (generic function with 1 method)

# Obtengo Bfators

In [6]:
# Leo nombres de structuras
tmp = readdlm(joinpath(home, "pdbs.list"))
pdbs_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));
aa = 277
aa_3 = aa * 3
aa_36 = aa_3 - 6
structs = length(pdbs_list)

# Leo activas e inactivas
tmp = readdlm(joinpath(home, "am.list"))
am_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

tmp = readdlm(joinpath(home, "im.list"))
im_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

cfg_filename = joinpath(home, "egfr.cfg")

"/home/german/labo/18/egfr/cavidad/egfr.cfg"

### Leo modos y frecuencias

In [21]:
proto_teo_bfactors = Array{Array{Float64, 1}}(length(pdbs_list))
modos = Array{Array{Float64, 2}}(length(pdbs_list))
freqs = Array{Array{Float64, 1}}(length(pdbs_list))

for i = 1:length(pdbs_list)
    each = pdbs_list[i]
    modos[i] = readdlm(joinpath(home_modos, each, string("modos_", each)))
    freqs[i] = convert(Array{Float64}, readdlm(joinpath(home_modos, each, string("freq_", each)))[8:end, 1])

end

### Obtengo Bfactors teóricos

In [29]:
mapslices(x -> sum(x), modos[i] .^ 2 * diagm(1 ./ freqs[i]), 2)

831×1 Array{Float64,2}:
 178.585 
 110.313 
  49.4086
  86.8141
  85.4585
  57.8547
 101.472 
  71.9284
  77.9335
  90.2357
 189.578 
 114.202 
 103.084 
   ⋮     
  34.6585
 115.892 
 103.466 
  67.7123
 112.817 
  94.7746
  46.2714
  61.0024
  69.6327
  45.4481
  50.6372
  68.4962

825×825 Array{Float64,2}:
 41892.7      0.0     0.0      0.0   …  0.0       0.0        0.0     
     0.0  12367.1     0.0      0.0      0.0       0.0        0.0     
     0.0      0.0  6784.29     0.0      0.0       0.0        0.0     
     0.0      0.0     0.0   4623.88     0.0       0.0        0.0     
     0.0      0.0     0.0      0.0      0.0       0.0        0.0     
     0.0      0.0     0.0      0.0   …  0.0       0.0        0.0     
     0.0      0.0     0.0      0.0      0.0       0.0        0.0     
     0.0      0.0     0.0      0.0      0.0       0.0        0.0     
     0.0      0.0     0.0      0.0      0.0       0.0        0.0     
     0.0      0.0     0.0      0.0      0.0       0.0        0.0     
     0.0      0.0     0.0      0.0   …  0.0       0.0        0.0     
     0.0      0.0     0.0      0.0      0.0       0.0        0.0     
     0.0      0.0     0.0      0.0      0.0       0.0        0.0     
     ⋮                               ⋱                          